In [1]:
import pygame
import random
import math
import queue
import pandas as pd
from datetime import datetime, timedelta, date
import time

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# ---------------- Global Configuration ----------------

PPM = 1100 / 170  # Pixels Per Meter (screen scaling)
pixel_speed = ((3 * PPM) / 60) * 10  # Converts 3 m/s to pixels per frame (adjusted for 60 FPS and visibility)

SIMULATION_SPEED = 10.0 # Adjust simulation speed here
update_log = []  # Global log for simulation messages

'''
# Set to store reservations that will have a longer wait time (4-5 minutes)
delayed_reservations = set()
# Counter to limit the number of delayed reservations
delayed_count = 0
# Maximum number of delayed reservations per flight#
MAX_DELAYED_RESERVATIONS = 0
'''

def add_update_message(message, color):
    update_log.append((message, color, pygame.time.get_ticks()))
    # Debug print
    print(f"[LOG] {message}")

"""Loads luggage data from CSV and filters for SLHS-handled bags."""
file_path = "~/Desktop/luggage_dataset5.csv"
df = pd.read_csv(file_path)
df = df[df['Cluster_Size'] != 1]
#df['Handled_by_SLHS'] = df['Handled_by_SLHS'].astype(str).str.lower()
#df = df[df['Handled_by_SLHS'] == 'true']

'''
def get_flights_data(data):
    """Organizes luggage data into a dictionary grouped by flight number."""
    flights = data['Flight_Number'].unique()
    flights_data = {}
    for flight in flights:
        flights_data[flight] = data[data['Flight_Number'] == flight]
    return flights_data

def expected_bags_for_reservation(reservation, fallback):
    """
    Returns the expected bag count extracted from the reservation id.
    For instance, if the id ends with "04", returns 4.
    Otherwise, uses the fallback value.
    """
    if reservation[-2:].isdigit():
        return int(reservation[-2:])
    return fallback'''

'\ndef get_flights_data(data):\n    """Organizes luggage data into a dictionary grouped by flight number."""\n    flights = data[\'Flight_Number\'].unique()\n    flights_data = {}\n    for flight in flights:\n        flights_data[flight] = data[data[\'Flight_Number\'] == flight]\n    return flights_data\n\ndef expected_bags_for_reservation(reservation, fallback):\n    """\n    Returns the expected bag count extracted from the reservation id.\n    For instance, if the id ends with "04", returns 4.\n    Otherwise, uses the fallback value.\n    """\n    if reservation[-2:].isdigit():\n        return int(reservation[-2:])\n    return fallback'

In [3]:
# ---------------- Definitions for Conveyor Belt and Key Points ----------------

conveyor_belts = [
    [(1200, 600), (100, 600)],
    [(100, 550), (800, 550)],
    [(200, 500), (800, 500)],
    [(200, 450), (700, 450)],
    [(300, 400), (700, 400)],
    [(300, 350), (600, 350)],
    [(100, 250), (900, 250)],
    [(100, 600), (100, 200)],
    [(200, 500), (200, 200)],
    [(300, 400), (300, 200)],
    [(600, 350), (600, 200)],
    [(700, 450), (700, 200)],
    [(800, 550), (800, 200)],
    [(900, 600), (900, 250)],
    [(580, 200), (600, 250)],
    [(620, 200), (600, 250)],
    [(320, 200), (300, 250)],
    [(280, 200), (300, 250)],
    [(680, 200), (700, 250)],
    [(720, 200), (700, 250)],
    [(180, 200), (200, 250)],
    [(220, 200), (200, 250)],
    [(780, 200), (800, 250)],
    [(820, 200), (800, 250)],
    [(80, 200), (100, 250)],
    [(120, 200), (100, 250)],
]

points = [
    (100, 250), (100, 550), (100, 600),
    (200, 250), (200, 450), (200, 500),
    (300, 250), (300, 350), (300, 400),
    (600, 250), (600, 350), (700, 250),
    (700, 400), (700, 450), (800, 250),
    (800, 500), (800, 550), (900, 250),
    (900, 600)
]

gates_entry_positions = [
    (600, 250),  # Gate 7
    (300, 250),  # Gate 6
    (700, 250),  # Gate 5
    (200, 250),  # Gate 4
    (800, 250),  # Gate 3
    (100, 250),  # Gate 2
]

pickup_gates_positions = [
    (600, 200), # Gates 2
    (580, 200),
    (620, 200),
    (300, 200), # Gates 3
    (320, 200),
    (280, 200),
    (700, 200), # Gates 4
    (680, 200),
    (720, 200),
    (200, 200), # Gates 5
    (180, 200),
    (220, 200),
    (800, 200), # Gates 6
    (780, 200), 
    (820, 200),
    (100, 200), # Gates 7
    (80, 200),
    (120, 200),
]



In [4]:
# ---------------- Path and Reservation Color Generation ----------------

class Path:
    def __init__(self, name, points):
        self.name = name
        self.points = points

paths = {
    7: Path("F", [(900, 600), (100, 600), (100, 550), (100, 250)]),
    6: Path("E", [(900, 600), (100, 600), (100, 550), (800, 550), (800, 500), (800, 250)]),
    5: Path("D", [(900, 600), (100, 600), (100, 550), (800, 550), (800, 500), (200, 500), (200, 450), (200, 250)]),
    4: Path("C", [(900, 600), (100, 600), (100, 550), (800, 550), (800, 500), (200, 500), (200, 450), (700, 450),
                   (700, 400), (700, 250)]),
    3: Path("B", [(900, 600), (100, 600), (100, 550), (800, 550), (800, 500), (200, 500), (200, 450), (700, 450),
                   (700, 400), (300, 400), (300, 350), (300, 250)]),
    2: Path("A", [(900, 600), (100, 600), (100, 550), (800, 550), (800, 500), (200, 500), (200, 450), (700, 450),
                   (700, 400), (300, 400), (300, 350), (600, 350), (600, 250)]),
    0: Path("Alaa", [(900, 600), (100, 600), (100, 550)]),
    17: Path("Woo", [(900, 600),(900, 600), (100, 600), (100, 550)])
}

paths_2 = {
    cluster: Path(str(cluster), points)
    for cluster, points in {
        7: [(900, 600), (100, 600), (100, 550), (800, 550), (800, 500), (200, 500),
            (200, 450), (700, 450), (700, 400), (300, 400), (300, 350), (600, 350),
            (600, 250), (900, 250), (900, 600)],
        6: [(900, 600), (100, 600), (100, 550), (800, 550), (800, 500), (200, 500),
            (200, 450), (700, 450), (700, 400), (300, 400), (300, 350), (600, 350),
            (600, 250), (900, 250), (900, 600)],
        5: [(900, 600), (100, 600), (100, 550), (800, 550), (800, 500), (200, 500),
            (200, 450), (700, 450), (700, 400), (300, 400), (300, 350), (600, 350),
            (600, 250), (900, 250), (900, 600)],
        4: [(900, 600), (100, 600), (100, 550), (800, 550), (800, 500), (200, 500),
            (200, 450), (700, 450), (700, 400), (300, 400), (300, 350), (600, 350),
            (600, 250), (900, 250), (900, 600)],
        3: [(900, 600), (100, 600), (100, 550), (800, 550), (800, 500), (200, 500),
            (200, 450), (700, 450), (700, 400), (300, 400), (300, 350), (600, 350),
            (600, 250), (900, 250), (900, 600)],
        2: [(900, 600), (100, 600), (100, 550), (800, 550), (800, 500), (200, 500),
            (200, 450), (700, 450), (700, 400), (300, 400), (300, 350), (600, 350),
            (600, 250), (900, 250), (900, 600)]
    }.items()
}

def generate_random_color():
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    
cluster_sizes = [2, 3, 4, 5, 6, 7, 8, 9, 10] 

cluster_colors = {size: generate_random_color() for size in cluster_sizes}

reservation_colors = {}
claim_stats = {}

In [5]:
# ---------------- Bag Class ----------------

class Bag:
    def __init__(self, bag_id, reservation_id, cluster_size, flight_number, flight_time):
        self.ready = False
        self.path = []
        self.bag_id = bag_id
        self.reservation = reservation_id
        self.flight_number = flight_number
        self.flight_time = flight_time
        self.cluster_size = cluster_size  # total number of bags in this reservation
        self.position = (1200, 600)
        self.assigned_to_gate = False
        self.gate = None
        self.speed = pixel_speed 
        self.target = None
        self.time = None
        self.at_gate = False
        self.route_points = [(100, 600), (100, 550), (800, 550), (800, 500), (200, 500), (200, 450), (700, 450), (700, 400), (300, 400), (300, 350), (600, 350), (600, 250), (900, 250), (900, 600)]
        self.current_point_index = 1
        self.color = cluster_colors[cluster_size]
        '''
        if self.reservation not in reservation_colors:
            reservation_colors[self.reservation] = generate_color_from_id(self.reservation)
        self.color = reservation_colors[self.reservation]
        self.last_reassign_attempt = 0'''

    def set_at_gate(self):
        self.at_gate = True

    def set_ready(self):
        self.ready = True   

    def set_path(self, path):
        self.path = path.points[:]
        if self.path:
            self.target = self.path.pop(0)

    def set_gate(self, pickup_gate):
        self.gate = pickup_gate

    def has_reached_gate(self):
        if self.gate is None:
            return False
        return self.position == self.gate.position

    def append_path(self, path):
        if isinstance(path, Path):  
            self.path.extend(path.points)  
        elif isinstance(path, list):  
            self.path.extend(path)  
        elif isinstance(path, tuple):
            self.path.append(path)
        
        if not self.target and self.path:
            self.target = self.path.pop(0) # Get it moving

    def append_remaining_path(self, path):
        for i, point in enumerate(path.points):
            if point == self.position:
                self.path = [self.position]
                self.append_path(path.points[i:])
                return True
            elif point == self.target:
                self.path = [self.target]
                self.append_path(path.points[i:])
                return True

        return False
                
                
    def move(self):
        if self.target:
            x, y = self.position
            tx, ty = self.target
    
            dx = tx - x
            dy = ty - y
            distance = (dx**2 + dy**2) ** 0.5
    
            if distance <= self.speed:
                self.position = self.target  
                if self.path:
                    self.target = self.path.pop(0)  
                else:
                    self.target = None  
            else:
                self.position = (x + (dx / distance) * self.speed, 
                                 y + (dy / distance) * self.speed)


In [6]:
# ---------------- Gate and PickupGate Classes ----------------
'''
gate_names = {7: "2", 6: "3", 5: "4", 4: "5", 3: "6", 2: "7"}
sub_gate_labels = ["A", "B", "C"]
terminal_priority = [7, 6, 5, 4, 3, 2]'''

class Gate:
    
    def __init__(self, name, gates, position):
        self.name = name
        self.gates = gates
        self.position = position

'''
    def search(self, bag):
        # First try to assign to a gate with the same reservation that is not full.
        for i, pickup in enumerate(self.gates):
            if pickup.reservation == bag.reservation and not pickup.full:
                return i
        # Otherwise, return the first unreserved gate.
        for i, pickup in enumerate(self.gates):
            if pickup.reservation is None:
                return i
        return -1'''

class PickupGate:
    
    def __init__(self, name, position, parent):
        self.name = name
        self.position = position
        self.parent = parent
        self.display_name = f"{name}"
        self.reservation = None
        self.load = 0
        self.bags = []
        self.full = False
        self.available = True
        self.dispense_time = None


    def set_reservation(self, reservation):
        if self.reservation is None:  
            self.reservation = reservation
            return True
        else:
            return False


    def add_luggage(self, bag):
        if bag.reservation == self.reservation:
            self.bags.append(bag)
            self.load += 1

    def is_full(self):
        if(self.load>0):
            if self.load == self.bags[0].cluster_size:
                return True
        return False

    def clear_luggage(self):
        removed_bags = self.bags if self.bags else []  
        self.reservation = None
        self.load = 0
        self.bags = [] 
        return removed_bags  
    

# ---------------- Initialize Gate Entries ----------------
gate_entries = []
k = 0
for i in range(6):
    gates = []
    for j in range(3):
        gates.append(PickupGate(j+1, pickup_gates_positions[k], i+2))
        k += 1
    gate_entries.append(Gate(i+2, gates, gates_entry_positions[i]))


In [7]:
point_to_gates = {

    (900, 250): [(600, 200), (580, 200), (620, 200),
                 (300, 200), (320, 200), (280, 200),
                 (700, 200), (680, 200), (720, 200),
                 (200, 200), (180, 200), (220, 200),
                 (800, 200), (780, 200), (820, 200),
                 (100, 200), (80, 200),  (120, 200)],
    
    (900, 600): [(600, 200), (580, 200), (620, 200),
                 (300, 200), (320, 200), (280, 200),
                 (700, 200), (680, 200), (720, 200),
                 (200, 200), (180, 200), (220, 200),
                 (800, 200), (780, 200), (820, 200),
                 (100, 200), (80, 200),  (120, 200)],

    (100, 600): [(600, 200), (580, 200), (620, 200),
                 (300, 200), (320, 200), (280, 200),
                 (700, 200), (680, 200), (720, 200),
                 (200, 200), (180, 200), (220, 200),
                 (800, 200), (780, 200), (820, 200),
                 (100, 200), (80, 200),  (120, 200)],

    (100, 550): [(600, 200), (580, 200), (620, 200),
                 (300, 200), (320, 200), (280, 200),
                 (700, 200), (680, 200), (720, 200),
                 (200, 200), (180, 200), (220, 200),
                 (800, 200), (780, 200), (820, 200),
                 (100, 200), (80, 200),  (120, 200)],

    (800, 550): [(600, 200), (580, 200), (620, 200),
                 (300, 200), (320, 200), (280, 200),
                 (700, 200), (680, 200), (720, 200),
                 (200, 200), (180, 200), (220, 200),
                 (800, 200), (780, 200), (820, 200)],
    
    (800, 500): [(600, 200), (580, 200), (620, 200),
                 (300, 200), (320, 200), (280, 200),
                 (700, 200), (680, 200), (720, 200),
                 (200, 200), (180, 200), (220, 200),
                 (800, 200), (780, 200), (820, 200)],
    
    (200, 500): [(600, 200), (580, 200), (620, 200),
                 (300, 200), (320, 200), (280, 200),
                 (700, 200), (680, 200), (720, 200),
                 (200, 200), (180, 200), (220, 200)],
    
    (200, 450): [(600, 200), (580, 200), (620, 200),
                 (300, 200), (320, 200), (280, 200),
                 (700, 200), (680, 200), (720, 200),
                 (200, 200), (180, 200), (220, 200)],
    
    (700, 450): [(600, 200), (580, 200), (620, 200),
                 (300, 200), (320, 200), (280, 200),
                 (700, 200), (680, 200), (720, 200)],
    
    (700, 400): [(600, 200), (580, 200), (620, 200),
                 (300, 200), (320, 200), (280, 200),
                 (700, 200), (680, 200), (720, 200)],

    (300, 400): [(600, 200), (580, 200), (620, 200),
                 (300, 200), (320, 200), (280, 200)],
    
    (300, 350): [(600, 200), (580, 200), (620, 200),
                 (300, 200), (320, 200), (280, 200)],

    (600, 350): [(600, 200), (580, 200), (620, 200)],

    (600, 250): [(600, 200), (580, 200), (620, 200)],

}

pickup_gates2 = []
for gate in gate_entries:
    pickup_gates2.extend(gate.gates) 


position_to_gate = {gate.position: gate for gate in pickup_gates2}

# Step 3: Convert your point_to_gates (positions) into point_to_gate_objects
points_to_gate = {
    point: [position_to_gate[pos] for pos in gate_positions if pos in position_to_gate]
    for point, gate_positions in point_to_gates.items()
}

In [8]:
def format_time(seconds):
    minutes = int(seconds // 60)
    sec = int(seconds % 60)
    return f"{minutes:02d}:{sec:02d}"

def convert_flight_time(flight_time_str):
    flight_time_obj = datetime.strptime(flight_time_str + ":00", '%H:%M:%S')
    return flight_time_obj

In [9]:
class Reservation:
    
    def __init__(self, id, cluster_size, flight_time, flight_number):
        self.id = id
        self.cluster_size = cluster_size
        self.flight_time = flight_time
        self.flight_number = flight_number
        self.time = None

    def __str__(self):
        return f"Reservation ID: {self.id}, Cluster Size: {self.cluster_size}, Time: {self.time}, Flight time: {self.flight_time}, Flight number: {self.flight_number}"

In [10]:
reservations = []

conveyor_belt = []

# Group bags by Flight Number
grouped_conveyor_belt = []

for flight_number, group in df.groupby("Flight_Number"):
    flight_bags = []
    
    flight_time = group.iloc[0]["Flight_Arrival_Time"]
    flight_start_time = convert_flight_time(flight_time) # Convert to milliseconds
    
    for reservation_id, res_group in group.groupby("Reservation_ID"):
        cluster_size = int(res_group.iloc[0]['Cluster_Size'])  # Get cluster size

        for _, row in res_group.iterrows():
            real_bag_id = row["Bag_ID"]  # Adjust this column name if different
            flight_bags.append(Bag(real_bag_id, reservation_id, cluster_size, flight_number, flight_time))

    # Shuffle bags within this flight
    random.shuffle(flight_bags)

    bag_time = flight_start_time  # Initialize the timestamp for the first bag

    for bag in flight_bags:
        # Assign timestamp after shuffling
        bag.time = datetime.combine(date.today(), bag_time.time())
        bag_time += timedelta(seconds=3)/10 

    # Optionally: store flight index to simulate 12-min delay later
    for bag in flight_bags:
        bag.flight_index = len(grouped_conveyor_belt)

    grouped_conveyor_belt.append(flight_bags)

for flight_group in grouped_conveyor_belt:
    conveyor_belt.extend(flight_group)

conveyor_belt = sorted(conveyor_belt, key=lambda bag: bag.time)

reservations_info = {}

for bag in conveyor_belt:
    if bag.reservation not in reservations_info:
        reservations_info[bag.reservation] = (bag.cluster_size, bag.flight_time, bag.flight_number)

reservations_f = {
    res_id: Reservation(res_id, cluster_size, flight_time, flight_number)
    for res_id, (cluster_size, flight_time, flight_number) in reservations_info.items()
}

reservations_c = []

In [11]:
# ---------------- Utility Functions for Drawing ----------------
def draw_log_box(screen, font):
    box_x, box_y, box_w, box_h = 920, 50, 450, 450
    bg_color = (250, 250, 250)
    BLACK = (0, 0, 0)
    GRAY = (80, 80, 80)
    pygame.draw.rect(screen, bg_color, (box_x, box_y, box_w, box_h))
    pygame.draw.rect(screen, GRAY, (box_x, box_y, box_w, box_h), 2)
    messages_to_show = update_log[-15:]
    text_y = box_y + 5
    for msg, color, _ in messages_to_show:
        text_surface = font.render(msg, True, BLACK)
        screen.blit(text_surface, (box_x + 20, text_y))
        pygame.draw.circle(screen, color, (box_x + 10, text_y + text_surface.get_height() // 2), 5)
        text_y += text_surface.get_height() + 5

def draw_gate_status_box(screen, medium_font, white_color):
    box_x, box_y, box_w, box_h = 920, 400, 450, 180
    bg_color = (250, 250, 250)
    BLACK = (0, 0, 0)
    GRAY = (80, 80, 80)
    pygame.draw.rect(screen, bg_color, (box_x, box_y, box_w, box_h))
    pygame.draw.rect(screen, GRAY, (box_x, box_y, box_w, box_h), 2)
    gate_status_y = box_y + 10
    gate_status_title = medium_font.render("Gate Status:", True, BLACK)
    screen.blit(gate_status_title, (box_x + 10, gate_status_y))
    gate_status_y += gate_status_title.get_height() + 5
    gates_per_line = 3
    gate_line_count = 0
    gate_col_count = 0
    gate_start_x = box_x + 10
    gate_current_x = gate_start_x
    gate_current_y = gate_status_y
    for gate in gate_entries:
        for pickup in gate.gates:
            if pickup.reservation is not None:
                remaining = pickup.expected_bags - pickup.load
                if remaining < 0: 
                    remaining = 0
                
                # Show if this is a delayed reservation
                if pickup.reservation in delayed_reservations:
                    status_str = f"{pickup.status} (DELAYED)"
                else:
                    status_str = f"{pickup.status} ({remaining})"
            else:
                status_str = pickup.status
            gate_line = f"{pickup.display_name}: {status_str}"
            gate_status_surface = medium_font.render(gate_line, True, BLACK)
            screen.blit(gate_status_surface, (gate_current_x + 10, gate_current_y))
            status_color = reservation_colors.get(pickup.reservation, white_color) if pickup.reservation else white_color
            pygame.draw.circle(screen, status_color, (gate_current_x, gate_current_y + gate_status_surface.get_height() // 2), 5)
            gate_current_x += 10 + gate_status_surface.get_width() + 40
            gate_col_count += 1
            if gate_col_count >= gates_per_line:
                gate_current_y += gate_status_surface.get_height() + 5
                gate_current_x = gate_start_x
                gate_col_count = 0
                gate_line_count += 1
            if gate_line_count >= 6:
                break
        if gate_line_count >= 6:
            break

def show_completion_screen(screen, width, height, title_font, subtitle_font, info_font, small_font, flight_time, return_on_key=True):
    completion_screen_time = 5  # seconds
    start_ticks = pygame.time.get_ticks()
    clock = pygame.time.Clock()
    WHITE = (255, 255, 255)
    BLACK = (0, 0, 0)
    BLUE = (65, 105, 225)
    GRAY = (200, 200, 200)
    while (pygame.time.get_ticks() - start_ticks) < completion_screen_time * 1000:
        screen.fill(WHITE)
        title_text = title_font.render("Flight Processing Complete", True, BLACK)
        screen.blit(title_text, (width // 2 - title_text.get_width() // 2, 100))
        #flight_text = subtitle_font.render(f"Flight: {flight_num}", True, BLUE)
        #screen.blit(flight_text, (width // 2 - flight_text.get_width() // 2, 160))
        stats_text = info_font.render("All bags claimed successfully", True, BLACK)
        screen.blit(stats_text, (width // 2 - stats_text.get_width() // 2, 220))
        time_text = info_font.render(f"Total processing time: {format_time(flight_time)}", True, BLACK)
        screen.blit(time_text, (width // 2 - time_text.get_width() // 2, 260))
        continue_text = small_font.render("Press any key to continue to next flight...", True, GRAY)
        screen.blit(continue_text, (width // 2 - continue_text.get_width() // 2, height - 100))
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return True
            elif event.type == pygame.KEYDOWN and return_on_key:
                return False
        pygame.display.update()
        clock.tick(30)
    return False

def show_waiting_screen(screen, width, height, title_font, subtitle_font, info_font, small_font, next_flight, wait_total=30):
    WHITE = (255, 255, 255)
    BLACK = (0, 0, 0)
    BLUE = (65, 105, 225)
    GRAY = (200, 200, 200)
    clock = pygame.time.Clock()
    wait_start = pygame.time.get_ticks()
    waiting = True
    while waiting:
        current_time = pygame.time.get_ticks()
        elapsed = (current_time - wait_start) / 1000.0
        remaining = wait_total - elapsed
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return True
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    waiting = False
        if remaining <= 0:
            break
        screen.fill(WHITE)
        title_text = title_font.render("Smart Luggage Handling System", True, BLACK)
        screen.blit(title_text, (width // 2 - title_text.get_width() // 2, 20))
        next_text = subtitle_font.render(f"Next Flight: {next_flight}", True, BLUE)
        screen.blit(next_text, (width // 2 - next_text.get_width() // 2, height // 2 - 60))
        wait_msg = subtitle_font.render("Waiting between flights:", True, BLACK)
        screen.blit(wait_msg, (width // 2 - wait_msg.get_width() // 2, height // 2 - 20))
        time_remaining = info_font.render(f"Time remaining: {format_time(remaining)}", True, BLACK)
        screen.blit(time_remaining, (width // 2 - time_remaining.get_width() // 2, height // 2 + 20))
        skip_msg = info_font.render("Press SPACE to skip waiting time", True, GRAY)
        screen.blit(skip_msg, (width // 2 - skip_msg.get_width() // 2, height // 2 + 60))
        pygame.display.update()
        clock.tick(10)
    return False



In [12]:
# ---------------- Main Simulation ----------------
pygame.init()

start_time = pygame.time.get_ticks()  
'''
def is_gate_in_use(pickup, scanned_bags):
    for b in scanned_bags:
        if b.gate == pickup and pickup.status == "Reserved":
            return True
    return False'''
    

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (65, 105, 225)
GRAY = (200, 200, 200)

font = pygame.font.Font(None, 20)
width, height = 1400, 720
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("Dijkstra's Algorithm Benchmark")
title_font = pygame.font.SysFont('Arial', 28, bold=True)
subtitle_font = pygame.font.SysFont('Arial', 20)
info_font = pygame.font.SysFont('Arial', 16)
medium_font = pygame.font.SysFont('Arial', 14)
small_font = pygame.font.SysFont('Arial', 12)
clock = pygame.time.Clock()

scanned_bags = []
bag_start_times = []  
i = 0
running = True
spawn_interval = 2000
last_spawn_time = 0
flight_start_time = pygame.time.get_ticks()

while running:
        
    screen.fill(WHITE)
    
    # Draw header and flight info.
    title_text = title_font.render("Dijkstra's Algorithm Benchmark", True, BLACK)
    screen.blit(title_text, (width // 2 - title_text.get_width() // 2, 20))
    #flight_text = subtitle_font.render(f"Flight: {flight_num}", True, BLUE)
    #screen.blit(flight_text, (width // 2 - flight_text.get_width() // 2, 60))
    elapsed_ms = (pygame.time.get_ticks() - start_time)*10
    elapsed_seconds = elapsed_ms // 1000
    minutes = elapsed_seconds // 60
    seconds = elapsed_seconds % 60
    time_text = font.render(f"Time: {minutes:02}:{seconds:02}", True, BLACK)
    screen.blit(time_text, (10, 10))
    
    # Draw conveyor belts and key points.
    for belt in conveyor_belts:
        pygame.draw.line(screen, BLACK, belt[0], belt[1], 3)
        
    for point in points:
        pygame.draw.circle(screen, BLACK, point, 5)
    
    # Draw gate entries and pickup gates.
    for gate in gate_entries:
        pygame.draw.rect(screen, BLACK, (gate.position[0] - 5, gate.position[1] - 5, 10, 10))
        for pickup in gate.gates:
            if pickup.reservation is None:
                pygame.draw.rect(screen, BLACK, (pickup.position[0] - 5, pickup.position[1] - 5, 10, 10))
            else:
                gate_color = reservation_colors.get(pickup.reservation, BLACK)
                pygame.draw.circle(screen, gate_color, pickup.position, 5)
            #point_label = small_font.render(pickup.display_name, True, BLACK)
            #screen.blit(point_label, (pickup.position[0] - 12, pickup.position[1] - 20))
    for gate_group in gate_entries: 
        for gate in gate_group.gates: 
            load_text = font.render(f"{gate.load}", True, BLACK)    
            screen.blit(load_text, (gate.position[0]-5, 177))
            
    help_text = small_font.render("Press ESC to exit | SPACE to skip wait time", True, GRAY)
    screen.blit(help_text, (width - 300, height - 20))
    
    #draw_log_box(screen, medium_font)
    #draw_gate_status_box(screen, medium_font, WHITE)
    
    # Event handling.
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    
    current_ticks = pygame.time.get_ticks()
    current_time = convert_flight_time(conveyor_belt[0].flight_time) + timedelta(milliseconds=current_ticks*10)
    current_time = current_time.time()

    more_items = len(conveyor_belt)-1

    if i < len(conveyor_belt):
        bag = conveyor_belt[i]
        if current_time >= bag.time.time():
            
            scanned_bags.append(bag)
            bag.set_ready()
            bag_start_times.append(bag.time.time())
             
    
            assigned = False        
            for gate in gate_entries:
                for pickup in gate.gates:
                    if pickup.available and not bag.assigned_to_gate:
                        bag.set_path(paths[gate.name])
                        bag.append_path(pickup.position)
                        bag.set_gate(pickup)
                        pickup.reservation = bag.reservation
                        pickup.available = False
                        bag.assigned_to_gate = True
                        assigned = True
                        for other_bag in conveyor_belt:
                            if other_bag.reservation == bag.reservation:
                                other_bag.set_path(paths[gate.name])
                                other_bag.append_path(pickup.position)
                                other_bag.set_gate(pickup)
                                other_bag.assigned_to_gate = True
                        break
                if assigned:
                    break  
                    
            if not bag.assigned_to_gate:
                bag.set_path(paths[0]) 
                
            i+=1

    still_moving = False 

    j=0
    for bag in scanned_bags:
 
        if bag.assigned_to_gate and bag.position == bag.gate.position and bag.at_gate:
            pygame.draw.circle(screen, bag.color, bag.position, 8)
        elif bag.assigned_to_gate and bag.position == bag.gate.position:
            bag.gate.add_luggage(bag)
            bag.set_at_gate()
            pygame.draw.circle(screen, bag.color, bag.position, 8)
        elif not bag.assigned_to_gate and bag.position == bag.route_points[bag.current_point_index]:            
            gates_to_check = points_to_gate.get(bag.position, [])
            for gate in gates_to_check:
                if gate.available:
                    if not bag.append_remaining_path(paths[gate.parent]):
                        bag.append_path(paths[gate.parent])
                        bag.append_path(gate.position)
                        bag.set_gate(gate)
                        gate.available = False
                        gate.reservation = bag.reservation
                        bag.assigned_to_gate = True
                    else:
                        bag.append_path(gate.position)
                        bag.set_gate(gate)
                        gate.available = False
                        gate.reservation = bag.reservation
                        bag.assigned_to_gate = True
                    if bag.path: 
                        bag.move()
                        pygame.draw.circle(screen, bag.color, bag.position, 8)
                    for other_bag in conveyor_belt: 
                        if not other_bag.assigned_to_gate and bag.reservation == other_bag.reservation and other_bag is not bag:
                            if other_bag.append_remaining_path(paths[gate.parent]):
                                other_bag.append_path(gate.position)
                                other_bag.set_gate(gate)
                                other_bag.assigned_to_gate = True
                            else:
                                other_bag.append_remaining_path(paths_2[2])
                                other_bag.append_path(paths[gate.parent])
                                other_bag.append_path(gate.position)
                                other_bag.set_gate(gate)
                                other_bag.assigned_to_gate = True
                    break

            if not bag.assigned_to_gate:
                bag.current_point_index = (bag.current_point_index + 1) % 14
                bag.append_path(bag.route_points[bag.current_point_index])
                if bag.path: 
                    bag.move()
                    pygame.draw.circle(screen, bag.color, bag.position, 8)
          
        elif current_time >= bag_start_times[j]:
            bag.move()
            pygame.draw.circle(screen, bag.color, bag.position, 8)
            still_moving = True 

        for gate_group in gate_entries:
            for gate in gate_group.gates:
                if gate.is_full():
                    res_id = gate.reservation
                    if res_id in reservations_f:  
                        reservations_f[res_id].time = current_time
                        reservations_c.append(reservations_f.pop(res_id))
                        
        for gate_group in gate_entries: 
            for gate in gate_group.gates: 
                if gate.is_full() and gate.dispense_time is None:
                    wait_seconds = random.randint(0, 70)/10
                    gate.dispense_time = datetime.combine(datetime.today(), current_time) + timedelta(seconds=wait_seconds)
                            

        for gate_group in gate_entries: 
            for gate in gate_group.gates: 
                if gate.dispense_time and current_time >= gate.dispense_time.time():
                    removed_bags = gate.clear_luggage()  
                    gate.available = True
                    gate.reservation = None
                    scanned_bags = [bag for bag in scanned_bags if bag not in removed_bags]  
                    print(f"Gate {gate_group.name} was emptied, and {len(removed_bags)} bags disappeared at {current_time}!")
                    gate.dispense_time = None  
        j+=1
        
    pygame.display.update()
    clock.tick(60) 

    if i >= len(conveyor_belt) and not still_moving:
        running = False  
        pygame.quit()
                        
         

2025-05-10 20:59:29.230 python[69794:4230112] +[IMKClient subclass]: chose IMKClient_Modern
2025-05-10 20:59:29.230 python[69794:4230112] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Gate 7 was emptied, and 3 bags disappeared at 13:31:08.600000!
Gate 7 was emptied, and 7 bags disappeared at 13:31:30.220000!
Gate 7 was emptied, and 6 bags disappeared at 13:31:48.080000!
Gate 5 was emptied, and 2 bags disappeared at 13:31:54.270000!
Gate 6 was emptied, and 4 bags disappeared at 13:32:29.220000!
Gate 4 was emptied, and 3 bags disappeared at 13:32:31.250000!
Gate 6 was emptied, and 4 bags disappeared at 13:32:34.040000!
Gate 6 was emptied, and 4 bags disappeared at 13:32:38.790000!
Gate 5 was emptied, and 3 bags disappeared at 13:32:41.610000!
Gate 5 was emptied, and 3 bags disappeared at 13:33:02.250000!
Gate 3 was emptied, and 3 bags disappeared at 13:33:07.650000!
Gate 5 was emptied, and 6 bags disappeared at 13:33:13.040000!
Gate 4 was emptied, and 7 bags disappeared at 13:33:16.040000!
Gate 4 was emptied, and 7 bags disappeared at 13:33:29.970000!
Gate 2 was emptied, and 3 bags disappeared at 13:33:34.440000!
Gate 2 was emptied, and 7 bags disappeared at 13:33:35.

In [13]:
for res in reservations_c:
    flight_time_obj = datetime.strptime(res.flight_time, "%H:%M" if len(res.flight_time) == 5 else "%H:%M:%S").time()

    served_dt = datetime.combine(datetime.today(), res.time)
    flight_dt = datetime.combine(datetime.today(), flight_time_obj)
    res.time = served_dt - flight_dt

    print(res)

Reservation ID: RES-00039, Cluster Size: 3, Time: 0:02:04.170000, Flight time: 13:29, Flight number: AA3472
Reservation ID: RES-00070, Cluster Size: 7, Time: 0:02:24.800000, Flight time: 13:29, Flight number: AA3472
Reservation ID: RES-00054, Cluster Size: 6, Time: 0:02:45.300000, Flight time: 13:29, Flight number: AA3472
Reservation ID: RES-00062, Cluster Size: 2, Time: 0:02:52.320000, Flight time: 13:29, Flight number: AA3472
Reservation ID: RES-00013, Cluster Size: 4, Time: 0:03:23.410000, Flight time: 13:29, Flight number: AA3472
Reservation ID: RES-00027, Cluster Size: 3, Time: 0:03:29.050000, Flight time: 13:29, Flight number: AA3472
Reservation ID: RES-00016, Cluster Size: 4, Time: 0:03:31.880000, Flight time: 13:29, Flight number: AA3472
Reservation ID: RES-00065, Cluster Size: 4, Time: 0:03:37.650000, Flight time: 13:29, Flight number: AA3472
Reservation ID: RES-00038, Cluster Size: 3, Time: 0:03:41.280000, Flight time: 13:29, Flight number: AA3472
Reservation ID: RES-00011, C